1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

'wget' is not recognized as an internal or external command,
operable program or batch file.


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [8]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('data/data_000637.txt',delimiter=',')[0:70000]
df


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
69995,1,1,13,3869200863,809,19
69996,1,0,61,3869200863,810,26
69997,1,0,18,3869200863,812,6
69998,1,0,57,3869200863,813,18


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [9]:
x = df["BX_COUNTER"].max()+1
print("The number of BX in a ORBIT is : ",x)

The number of BX in a ORBIT is :  3564


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [11]:
conversion = 25*10**(-9)
deltatime = (df['ORBIT_CNT'].max()-df['ORBIT_CNT'].min())*x*conversion
print('Time: ', deltatime)

Time:  0.0620136


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [12]:
df = df.groupby(by='TDC_CHANNEL', sort=True).count()
df = df.sort_values('HEAD', ascending=False)
top = [x for x in df.index[0:3]]
print("top 3 TDC channels are ", top)

top 3 TDC channels are  [139, 64, 63]


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [14]:
print('The number of non-empty orbits is ' ,df['ORBIT_CNT'].nunique())


The number of non-empty orbits is  116


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [15]:
print('The number of unique orbits with at least one measurement from TDC_CHANNEL=139 is ' ,df[df['TDC_CHANNEL'] == 139]['ORBIT_CNT'].nunique())

KeyError: 'TDC_CHANNEL'

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.